In [15]:
import pymongo
from urllib.parse import quote_plus
from bson import SON
from bson import SON

In [8]:

class MongoConfig:
    def __init__(self, host_uri: str, port: int, user: str, password: str, database: str = None):
        self.host_uri = host_uri
        self.port = port
        self.user = user
        self.password = password
        self.database = database

    def connect(self) -> pymongo.MongoClient:
        connection_uri = f"mongodb://{quote_plus(self.user)}:{quote_plus(self.password)}@{self.host_uri}:{str(self.port)}"
        return pymongo.MongoClient(connection_uri)


def GetConfig():
    return MongoConfig(
        host_uri="localhost",
        port=27017,
        user="root",
        password="example"
    ).connect()


In [12]:
mongo_client = GetConfig()
mongo_db = mongo_client["test"]
mongo_collection = mongo_db["testcol"]

In [22]:
# 
test_user = {
		"@id": "ark:99999/test-user1",
		"name": "Test User1",
		"type": "Person",
		"email": "testuser1@example.org",
		"password": "test1",
		"organizations": [],
		"projects": [],
		"datasets": [],
		"software": [],
		"computations": [],
		"evidencegraphs": []
		}



test_dataset = {
	"@id": "ark:99999/test-dataset",
	"@type": "evi:Dataset",
	"name": "test dataset",
	"owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}
}

test_data_download = {
	"@id": "ark:99999/test-download",
	"name": "test file.txt",
	"@type": "DataDownload",
	"encodingFormat": ".txt",
	"encodesCreativeWork": test_dataset["@id"],
}

In [25]:
# clear all documents in the 
result = mongo_collection.delete_many({})

In [26]:
result.deleted_count

3

In [27]:
mongo_collection.insert_many([test_user, test_dataset])

In [28]:
create_download_bulk_write = [
	# update the dataset containing this download
	pymongo.UpdateOne(
		{"@id": test_dataset.get("@id")}, 
		{"$addToSet" : { 
			"distribution": SON([("@id", test_data_download.get("@id") ), ("@type", "DataDownload"), ("name", test_data_download.get("name"))])}
		}),

	# create metadata record in mongo
	pymongo.InsertOne(test_data_download)
]

bulk_write_result = mongo_collection.bulk_write(create_download_bulk_write)

#except Exception as bwe:
#	return OperationStatus(False, f"create download error: {bwe}", 500)

In [10]:
mongo_client.close()